In [ ]:
import pandas as pd
from numpy.random import choice
from my_pickle import unpickle_it
import matplotlib.pyplot as plt

In [ ]:
user_df = unpickle_it('user_df')
convo_df = unpickle_it('convo_df')

print("Make sure we're not useing ECT data")
print(pd.to_datetime(convo_df.timestamp.max()*1000000))

# CELL TO BE COPY-PASTED INTO .PY FILE

In [ ]:
def feature_time(df, user_df):

    ad,sg,ro,sr,sc,sn,ss,sa,rs,t = [],[],[],[],[],[],[],[],[],[]

    c=0
    for index, row in df.iterrows():
        try:
            first = user_df.loc[str(row.first_uid)]
            second = user_df.loc[str(row.second_uid)]
            ad.append(abs(first.age - second.age))
            sg.append(first.gender == second.gender)
            ro.append(get_rent_range(first, second))
            sr.append(first.inRelationship == second.inRelationship)
            sc.append(first.isClean == second.isClean)
            sn.append(first.isNight == second.isNight)
            ss.append(first.isStudent == second.isStudent)
            sa.append(first.attractiveness)
            rs.append(second.selectivity)
            t.append(False)
        except:
            ad.append('trouble')
            sg.append('trouble')
            ro.append('trouble')
            sr.append('trouble')
            sc.append('trouble')
            sn.append('trouble')
            ss.append('trouble')
            sa.append('trouble')
            rs.append('trouble')
            t.append(True)
            c += 1

    df['age_dif'] = ad
    df['same_gender'] = sg
    df['rent_overlap'] = ro
    df['same_relate'] = sr
    df['same_clean'] = sc
    df['same_night'] = sn
    df['same_student'] = ss
    df['sender_attractiveness'] = sa
    df['receiver_selectivity'] = rs
    df['trouble'] = t

    df = df.drop(df[df.trouble].index)
    df = df.drop(['trouble'], axis=1)
    print("Num bad rows: {}".format(c))
    return df

In [ ]:
X = feature_time(convo_df, user_df)
X.head(100)

In [ ]:
def get_rent_range(first_user, second_user):
    max1 = first_user.maxCost
    max2 = second_user.maxCost
    min1 = first_user.minCost
    min2 = second_user.minCost
    if max1*max2*min1*min2 > 0:
        upper = min(first_user.maxCost,second_user.maxCost)
        lower = max(first_user.minCost,second_user.minCost)
        if upper-lower > 0: return upper-lower
        else: return 0
    else: return 175

In [ ]:
yes1 = '0a9yOPKFSH'
yes2 = '013LzOrVju'
no1 = '0UBLgJIHgz'
no2 = '0EWNOmyQmW'
low = '01DE0NCjwh'
high = '02GDyQPLII'
print(get_rent_range(user_df.loc[yes1], user_df.loc[yes2]))
print(get_rent_range(user_df.loc[yes1], user_df.loc[no1]))
print(get_rent_range(user_df.loc[no2], user_df.loc[no1]))
print(get_rent_range(user_df.loc[low], user_df.loc[high]))

In [ ]:
user_df['cost_range'] = user_df.maxCost-user_df.minCost
user_df.cost_range.hist(bins=range(0,2000,50))
plt.show()
user_df.head(2)

In [ ]:
def get_average_overlap(df,samples):
    # to find average overlap of two users' rent ranges
    # comes out to about $175
    s = 0
    for n in range(samples):
        two_users = choice(user_df[user_df.cost_range>0].index,2)
        overlap = user_df.loc[two_users,'maxCost'].min()-user_df.loc[two_users,'minCost'].max()
        if overlap < 0: overlap = 0
        s += overlap
    return s/samples

In [ ]:
X = feature_time(convo_df, user_df)

In [ ]:
print(len(X))